In [4]:
import advent

def process(char):
    return {'.': 0, 'O': 1, '#': 2}[char]

data = advent.get_char_grid(14, 'txt', process)


In [5]:
import scipy
import numpy as np

dirdict = {
    'NORTH': (1, 7),
    'SOUTH': (7, 1),
    'WEST': (3, 5),
    'EAST': (5, 3)
}

def fall(filter: list[int], direction: str='NORTH'):
    # input is list with 9 elements that looks like:
    # 0 1 2
    # 3 4 5
    # 6 7 8
    # We return the content of cell 4 after a step

    # NOTE: this is a half step. E.g.
    # OOO. would be processed as OOO.->OO.O->O.OO->.OOO

    # NOTE: the 'direction' variable was added in part 2, before that it was just NORTH
    match filter[4]:
        case 0:
            if filter[dirdict[direction][1]] == 1: return 1 # O falls up
            else: return 0 # Nothing to fall
        case 1:
            if filter[dirdict[direction][0]] == 0: return 0 # O falls up
            else: return 1 # O stays still
        case 2: return 2 # # stays still
        case _: raise ValueError("incorrect filter value")

def full_fall(grid, direction: str='NORTH'):
    previous_hash, hash = 0, 1

    _fall = lambda f: fall(f, direction)

    while previous_hash != hash:
        # VERY IMPORTANT! we need the cval to be 2 (#) so the O dont fall out the bottom
        grid = scipy.ndimage.generic_filter(grid, _fall, (3, 3), mode='constant', cval=2)
        # Amazing 'hashing' function, found on stackoverflow
        previous_hash, hash = hash, grid.data.tobytes()
    return grid


def score(grid):
    # flip the grid so the stones are at the bottom
    rows = np.where(np.flip(grid, 0) == 1)[0]
    # This assignment starts counting rows at 1
    return sum(row + 1 for row in rows)

score(full_fall(data.copy()))

108840

In [9]:
# Part 2 approach: cycle detection baby!
from tqdm import tqdm

def step(grid):
    grid = full_fall(grid, 'NORTH')
    grid = full_fall(grid, 'WEST')
    grid = full_fall(grid, 'SOUTH')
    grid = full_fall(grid, 'EAST')
    return grid

def n_steps(grid, n: int):
    for _ in range(n): grid = step(grid)
    return grid

steps = 0
grid = data.copy()
cache = {grid.data.tobytes(): 0}
with tqdm(total = 1000) as pbar:
    while True:
        grid = step(grid)
        steps += 1
        hash = grid.data.tobytes()
        if hash in cache: break
        cache[hash] = steps
        pbar.update(steps)

17020it [01:24, 200.37it/s]                       


In [25]:
# We now have a cycle:
cycle_length = steps - cache[hash]
cycle_start = cache[hash]
cycle_end = (1_000_000_000 - cycle_start) % cycle_length
solution_val = cycle_start + cycle_end
print(cycle_start, cycle_length, (1_000_000_000 - cycle_start) % cycle_length, solution_val)

# Get the grid from the cache again, and turn the bytes back into a grid
# (this is why you dont use hashes when you can use the data itself as a hash :)
solution_bytes = [c for c in cache if cache[c] == solution_val][0]
grid_reformed = np.frombuffer(solution_bytes, dtype=grid.dtype, like=grid).reshape(grid.shape)

score(grid_reformed)

126 59 24 150


103445